# Step 1: Read Delta table from Lakehouse

In [4]:
emp_df = spark.read.format("delta").table("emp")

emp_df.show(5)


StatementMeta(, 595d1567-7e12-4efa-bb88-6fe3238f5960, 6, Finished, Available, Finished)

+-----+------+--------+----+---------+----+----+------+
|empno| ename|     job| mgr| hiredate| sal|comm|deptno|
+-----+------+--------+----+---------+----+----+------+
| 7369| SMITH|   CLERK|7902|17-Dec-80| 800|NULL|    20|
| 7900| JAMES|   CLERK|7698| 3-Dec-81| 950|NULL|    30|
| 7876| ADAMS|   CLERK|7788|23-May-87|1000|NULL|    20|
| 7521|  WARD|SALESMAN|7698|22-Feb-81|1250| 500|    30|
| 7654|MARTIN|SALESMAN|7698|28-Sep-81|1250|1400|    30|
+-----+------+--------+----+---------+----+----+------+
only showing top 5 rows



# Step 2: Perform Transformations

In [5]:
from pyspark.sql.functions import col, coalesce, lit, when

emp_transformed_df = (
    emp_df
    .withColumn("sal", coalesce(col("sal"), lit(0)))
    .withColumn("comm", coalesce(col("comm"), lit(0)))
    .withColumn("total_salary", col("sal") + col("comm"))
    .withColumn(
        "salary_grade",
        when(col("total_salary") >= 10000, "HIGH")
        .when(col("total_salary") >= 5000, "MEDIUM")
        .otherwise("LOW")
    )
)

emp_transformed_df.show(5)


StatementMeta(, 595d1567-7e12-4efa-bb88-6fe3238f5960, 7, Finished, Available, Finished)

+-----+------+--------+----+---------+----+----+------+------------+------------+
|empno| ename|     job| mgr| hiredate| sal|comm|deptno|total_salary|salary_grade|
+-----+------+--------+----+---------+----+----+------+------------+------------+
| 7369| SMITH|   CLERK|7902|17-Dec-80| 800|   0|    20|         800|         LOW|
| 7900| JAMES|   CLERK|7698| 3-Dec-81| 950|   0|    30|         950|         LOW|
| 7876| ADAMS|   CLERK|7788|23-May-87|1000|   0|    20|        1000|         LOW|
| 7521|  WARD|SALESMAN|7698|22-Feb-81|1250| 500|    30|        1750|         LOW|
| 7654|MARTIN|SALESMAN|7698|28-Sep-81|1250|1400|    30|        2650|         LOW|
+-----+------+--------+----+---------+----+----+------+------------+------------+
only showing top 5 rows



# Step 3: Write back to Lakehouse as Delta table

In [6]:
emp_transformed_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("emp_enriched_delta")


StatementMeta(, 595d1567-7e12-4efa-bb88-6fe3238f5960, 8, Finished, Available, Finished)

# Step 4: Validate the output

In [7]:
spark.sql("SELECT * FROM emp_enriched_delta").show()


StatementMeta(, 595d1567-7e12-4efa-bb88-6fe3238f5960, 9, Finished, Available, Finished)

+-----+------+---------+----+---------+----+----+------+------------+------------+
|empno| ename|      job| mgr| hiredate| sal|comm|deptno|total_salary|salary_grade|
+-----+------+---------+----+---------+----+----+------+------------+------------+
| 7369| SMITH|    CLERK|7902|17-Dec-80| 800|   0|    20|         800|         LOW|
| 7900| JAMES|    CLERK|7698| 3-Dec-81| 950|   0|    30|         950|         LOW|
| 7876| ADAMS|    CLERK|7788|23-May-87|1000|   0|    20|        1000|         LOW|
| 7521|  WARD| SALESMAN|7698|22-Feb-81|1250| 500|    30|        1750|         LOW|
| 7654|MARTIN| SALESMAN|7698|28-Sep-81|1250|1400|    30|        2650|         LOW|
| 7934|MILLER|    CLERK|7782|23-Jan-82|1300|   0|    10|        1300|         LOW|
| 7844|TURNER| SALESMAN|7698| 8-Sep-81|1500|   0|    30|        1500|         LOW|
| 7499| ALLEN| SALESMAN|7698|20-Feb-81|1600| 300|    30|        1900|         LOW|
| 7782| CLARK|  MANAGER|7839| 9-Jun-81|2450|   0|    10|        2450|         LOW|
| 76